In [1]:
# Some imports, we are not gong to use all the imports in this workbook but in subsequent workbooks we surely will.
import os
import time

import numpy as np
import pandas as pd

from toolz import pipe as p
import tensorflow as tf


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
import model_runs
import models as models
import utils as u

from importlib import reload
reload(u)
reload(model_runs)
reload(models)

Using TensorFlow backend.


<module 'models' from 'C:\\Users\\jod204\\projects\\jo\\samplesize_test\\models.py'>

# Settings and Helpers

In [3]:
# Define some Global Variables
max_features = 20000 # Maximum Number of words we want to include in our dictionary
maxlen = 400 # No of words in question we want to create a sequence with
embed_size = 50# Size of word to vec embedding we are using


def get_log_dir(iteration, samples_per_class):
    return f"output/{samples_per_class}/{iteration}/"


def save_series_gen(log_dir, prefix):
    def save_series(series, name):
        series.to_csv(f'{log_dir}/{prefix}_{name}.csv', index = False, header = False)
    
    return save_series


def log_truths(log_dir, train_y, val_y, test_y):
    save_series = save_series_gen(log_dir, 'true')
        
    save_series(train_y, 'train')
    save_series(val_y, 'val')
    save_series(test_y, 'test')


def log_iteration_preds(log_dir, model_name,
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time):
    def save_arr(arr, name):
        arr.tofile(f'{log_dir}/{model_name}_{name}.csv', sep = ',')
    
    save_arr(pred_train_y, 'pred_train')
    save_arr(pred_val_y, 'pred_val')
    save_arr(pred_test_y, 'pred_test')
    p(one_train_time,
      np.array,
      lambda _: save_arr(_, 'time'))


def run_and_log_model(model, model_name, train_X, train_y, val_X, val_y, test_X, iteration, samples_per_class):
    start_time = time.perf_counter()
    pred_train_y, pred_val_y, pred_test_y = model_runs.train_pred(model, train_X, train_y,
                                          val_X, val_y, test_X, epochs=50)
    end_time = time.perf_counter()
    one_train_time = end_time - start_time
    
    dir_name = get_log_dir(iteration, samples_per_class)
    log_iteration_preds(dir_name, model_name, 
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time) 


# Model Execution

In [ ]:
if __name__ == '__main__':
  with tf.device('/device:GPU:0'):
      data = pd.read_csv('./content/pubmed_cr_hep_ctl_abstracts_clean.csv')

      for samples_per_class in range(500, 2500, 500):
          for i in range(0, 25):
              train_X, val_X, test_X, train_y, val_y, test_y, word_index = u.load_and_prec(
                  samples_per_class, max_features, maxlen = maxlen, data = data)
            
              log_dir = get_log_dir(i, samples_per_class)
              os.makedirs(log_dir, exist_ok = True)
            
              log_truths(log_dir, train_y, val_y, test_y)
              
              print(test_X.shape[1])
              sequence_len = test_X.shape[1]
                
              def run_and_log_model_curry(model, model_name):
                run_and_log_model(model, model_name, 
                                  train_X, train_y, val_X, val_y, test_X, 
                                  i, samples_per_class)

              # CNN
              cnn_model_built = models.cnn_model(sequence_len, [3, 4, 5], maxlen, max_features, embed_size, 
                                                 num_filters=200, drop_rate=0.2)
              cnn_model_built.summary()
              run_and_log_model_curry(cnn_model_built, 'cnn')

              # LSTM
              lstm_model = models.model_lstm_du(maxlen, max_features, embed_size)
              lstm_model.summary()
              run_and_log_model_curry(lstm_model, 'lstm')

              # HAN
              han_model = models.model_lstm_atten(maxlen, max_features, embed_size)
              han_model.summary()
              run_and_log_model_curry(han_model, 'han')


1000
500
500
X shape :  (1000,)
y shape :  (1000,)
400
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 50)      1000000     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 400, 50, 1)   0           embedding_1[0][0]                
_________________________________________________________________

Epoch 29/50
603/603 [==============================] - 0s 180us/step - loss: 0.2616 - acc: 0.9701 - val_loss: 0.3575 - val_acc: 0.9254
Epoch 30/50
603/603 [==============================] - 0s 183us/step - loss: 0.2496 - acc: 0.9735 - val_loss: 0.3487 - val_acc: 0.9254
Epoch 31/50
603/603 [==============================] - 0s 180us/step - loss: 0.2388 - acc: 0.9768 - val_loss: 0.3413 - val_acc: 0.9403
Epoch 32/50
603/603 [==============================] - 0s 162us/step - loss: 0.2302 - acc: 0.9801 - val_loss: 0.3355 - val_acc: 0.9403
Epoch 33/50
603/603 [==============================] - 0s 202us/step - loss: 0.2225 - acc: 0.9818 - val_loss: 0.3316 - val_acc: 0.9403
Epoch 34/50
603/603 [==============================] - 0s 182us/step - loss: 0.2137 - acc: 0.9784 - val_loss: 0.3286 - val_acc: 0.9403
Epoch 35/50
603/603 [==============================] - 0s 163us/step - loss: 0.2073 - acc: 0.9818 - val_loss: 0.3240 - val_acc: 0.9403
Epoch 36/50
603/603 [==============================] - 

603/603 [==============================] - 7s 12ms/step - loss: 0.6932 - acc: 0.4992 - val_loss: 0.6944 - val_acc: 0.4328
Epoch 2/50
603/603 [==============================] - 4s 6ms/step - loss: 0.6929 - acc: 0.5075 - val_loss: 0.6928 - val_acc: 0.6119
Epoch 3/50
603/603 [==============================] - 4s 6ms/step - loss: 0.6927 - acc: 0.5920 - val_loss: 0.6914 - val_acc: 0.5672
Epoch 4/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6927 - acc: 0.4942 - val_loss: 0.6917 - val_acc: 0.5970

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6915 - acc: 0.6036 - val_loss: 0.6933 - val_acc: 0.4627

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 6/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6908 - acc: 0.5456 - val_loss: 0.6943 - val_acc: 0.4328

Epoch 00006: ReduceLROnPlateau reducing learning rate t

603/603 [==============================] - 0s 166us/step - loss: 0.3117 - acc: 0.9701 - val_loss: 0.3911 - val_acc: 0.9254
Epoch 26/50
603/603 [==============================] - 0s 192us/step - loss: 0.2961 - acc: 0.9768 - val_loss: 0.3809 - val_acc: 0.9254
Epoch 27/50
603/603 [==============================] - 0s 192us/step - loss: 0.2825 - acc: 0.9784 - val_loss: 0.3716 - val_acc: 0.9254
Epoch 28/50
603/603 [==============================] - 0s 177us/step - loss: 0.2695 - acc: 0.9801 - val_loss: 0.3637 - val_acc: 0.9254
Epoch 29/50
603/603 [==============================] - 0s 192us/step - loss: 0.2556 - acc: 0.9784 - val_loss: 0.3573 - val_acc: 0.9254
Epoch 30/50
603/603 [==============================] - 0s 192us/step - loss: 0.2457 - acc: 0.9784 - val_loss: 0.3520 - val_acc: 0.9254
Epoch 31/50
603/603 [==============================] - 0s 166us/step - loss: 0.2362 - acc: 0.9784 - val_loss: 0.3463 - val_acc: 0.9254
Epoch 32/50
603/603 [==============================] - 0s 190us/ste

603/603 [==============================] - 2s 3ms/step - loss: 0.2894 - acc: 0.8839 - val_loss: 0.7374 - val_acc: 0.6418

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 17/50
603/603 [==============================] - 2s 3ms/step - loss: 0.3103 - acc: 0.8640 - val_loss: 0.6983 - val_acc: 0.6567

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 18/50
603/603 [==============================] - 2s 3ms/step - loss: 0.2845 - acc: 0.8756 - val_loss: 0.7234 - val_acc: 0.6418

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 19/50
603/603 [==============================] - 2s 3ms/step - loss: 0.2709 - acc: 0.8823 - val_loss: 0.7201 - val_acc: 0.6418
Epoch 00019: early stopping
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 400)               0         
__________________________

...

Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 318s 176ms/step - loss: 0.6929 - acc: 0.4997 - val_loss: 0.6914 - val_acc: 0.5174
Epoch 2/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6912 - acc: 0.5097 - val_loss: 0.6895 - val_acc: 0.5622
Epoch 3/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6886 - acc: 0.5616 - val_loss: 0.6861 - val_acc: 0.6219
Epoch 4/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6838 - acc: 0.7446 - val_loss: 0.6792 - val_acc: 0.7910
Epoch 5/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6726 - acc: 0.8436 - val_loss: 0.6611 - val_acc: 0.8557
Epoch 6/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6409 - acc: 0.8850 - val_loss: 0.5991 - val_acc: 0.8756
Epoch 7/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.5190 - acc: 0.8701 - val_loss: 0.4576 - val_acc: 0.8109
Epoch 8/50


Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 316s 175ms/step - loss: 0.7376 - acc: 0.4964 - val_loss: 0.7099 - val_acc: 0.5672
Epoch 2/50
1809/1809 [==============================] - 1s 296us/step - loss: 0.7042 - acc: 0.5694 - val_loss: 0.6908 - val_acc: 0.8209
Epoch 3/50
1809/1809 [==============================] - 1s 286us/step - loss: 0.6853 - acc: 0.8939 - val_loss: 0.6756 - val_acc: 0.9453
Epoch 4/50
1809/1809 [==============================] - 1s 286us/step - loss: 0.6657 - acc: 0.9348 - val_loss: 0.6548 - val_acc: 0.9453
Epoch 5/50
1809/1809 [==============================] - 1s 289us/step - loss: 0.6360 - acc: 0.9403 - val_loss: 0.6244 - val_acc: 0.9453
Epoch 6/50
1809/1809 [==============================] - 1s 286us/step - loss: 0.5957 - acc: 0.9420 - val_loss: 0.5878 - val_acc: 0.9453
Epoch 7/50
1809/1809 [==============================] - 1s 286us/step - loss: 0.5489 - acc: 0.9464 - val_loss: 0.5472 - val_acc: 0.9453

Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 326s 180ms/step - loss: 0.6927 - acc: 0.5030 - val_loss: 0.6891 - val_acc: 0.5672
Epoch 2/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6900 - acc: 0.5069 - val_loss: 0.6861 - val_acc: 0.5771
Epoch 3/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6860 - acc: 0.5312 - val_loss: 0.6820 - val_acc: 0.5920
Epoch 4/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6787 - acc: 0.6274 - val_loss: 0.6735 - val_acc: 0.7562
Epoch 5/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6628 - acc: 0.8441 - val_loss: 0.6496 - val_acc: 0.8109
Epoch 6/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6237 - acc: 0.8248 - val_loss: 0.6098 - val_acc: 0.7612
Epoch 7/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.4801 - acc: 0.8900 - val_loss: 0.4485 - val_acc: 0.8010
Epoch 8/50


Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 330s 182ms/step - loss: 0.7378 - acc: 0.5141 - val_loss: 0.7121 - val_acc: 0.7264
Epoch 2/50
1809/1809 [==============================] - 1s 322us/step - loss: 0.7038 - acc: 0.6871 - val_loss: 0.6906 - val_acc: 0.8557
Epoch 3/50
1809/1809 [==============================] - 1s 298us/step - loss: 0.6846 - acc: 0.9099 - val_loss: 0.6723 - val_acc: 0.9353
Epoch 4/50
1809/1809 [==============================] - 1s 295us/step - loss: 0.6644 - acc: 0.9259 - val_loss: 0.6478 - val_acc: 0.9453
Epoch 5/50
1809/1809 [==============================] - 1s 297us/step - loss: 0.6355 - acc: 0.9342 - val_loss: 0.6140 - val_acc: 0.9403
Epoch 6/50
1809/1809 [==============================] - 1s 286us/step - loss: 0.5964 - acc: 0.9364 - val_loss: 0.5733 - val_acc: 0.9403
Epoch 7/50
1809/1809 [==============================] - 1s 303us/step - loss: 0.5507 - acc: 0.9359 - val_loss: 0.5310 - val_acc: 0.9403

Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 339s 187ms/step - loss: 0.6930 - acc: 0.5053 - val_loss: 0.6921 - val_acc: 0.5124
Epoch 2/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6909 - acc: 0.6059 - val_loss: 0.6905 - val_acc: 0.6219
Epoch 3/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6878 - acc: 0.6075 - val_loss: 0.6873 - val_acc: 0.5572
Epoch 4/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6820 - acc: 0.6993 - val_loss: 0.6794 - val_acc: 0.8308
Epoch 5/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6688 - acc: 0.8706 - val_loss: 0.6600 - val_acc: 0.7861
Epoch 6/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.6369 - acc: 0.8762 - val_loss: 0.6018 - val_acc: 0.8806
Epoch 7/50
1809/1809 [==============================] - 5s 3ms/step - loss: 0.5145 - acc: 0.9088 - val_loss: 0.3457 - val_acc: 0.8806
Epoch 8/50
